In [3]:
!pip install geopandas
!pip install backoff
!pip install statsmodels
!pip install seaborn
!pip install scipy
!pip install mapclassify
!pip install unidecode
!pip install rasterio
!pip install seaborn
!pip install mpl_scatter_density
!pip install rasterstats
!pip install xmltodict
!pip install pandas

In [4]:
import os
import subprocess
import geopandas
import pandas as pd
import json
import datetime
import backoff
import xmltodict
import fileinput
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
import fileinput
import glob
import matplotlib.pyplot as plt
# import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import scipy.stats
import scipy.stats as stats
from statsmodels import graphics
import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from unidecode import unidecode
import rasterio
from rasterio.plot import show
import seaborn
# import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors
from textwrap import wrap
import matplotlib.patches as mpatches
import pylab as py
import rasterstats
from rasterstats import zonal_stats
import scipy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import subprocess
import glob
import shutil
import time

from maap.maap import MAAP
# maap=MAAP(maap_host="api.ops.maap-project.org") 
maap=MAAP() 

def BUILD_TABLE_JOBSTATUS(submit_results_df, status_col = 'status'):
    import xmltodict
    
    # If jobs failed to submit, then they have a NaN for jobid, which makes the merge (join) fail
    submit_results_df = submit_results_df.fillna('')
    
    job_status_df = pd.concat([pd.DataFrame({'job_id': [job_id], 'status':[maap.getJobStatus(job_id)]}) for job_id in submit_results_df.job_id.to_list()])
    job_status_df = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='job_id')
    
    print(f'Count total jobs:\t{len(job_status_df)}')
    print(f"Count pending jobs:\t{job_status_df[job_status_df[status_col] =='Accepted'].shape[0]}")
    print(f"Count running jobs:\t{job_status_df[job_status_df[status_col] =='Running'].shape[0]}")
    
    NUM_FAILS = job_status_df[job_status_df[status_col] =='Failed'].shape[0]
    NUM_SUCCEEDS = job_status_df[job_status_df[status_col] =='Succeeded'].shape[0]
    print(f"Count succeeded jobs:\t{NUM_SUCCEEDS}")
    print(f"Count failed jobs:\t{NUM_FAILS}")
    if NUM_FAILS > 0:
        print(f"% of failed jobs:\t{round(NUM_FAILS / ( NUM_FAILS + NUM_SUCCEEDS ), 4) * 100}\n")
    else:
        print(f"% of failed jobs:\tNothing has failed...yet\n")
    
    return job_status_df

### DPS Jobs for all TILES

#### COMMIT ALL CHANGES and REGISTER ALGORITHM FROM TERMINAL

In [5]:
# cd /projects/ADE_biomass_harmonization/NASA_CMS
# git add NASA_CMS_2023/Hunka_2024_NSD/Layer_Alignment
# git commit -m 'Aligining layers to 10 x 10 degree grid'
# git tag -f run_layer_alignment_v1

# git push origin -f run_layer_alignment_v1
# git push
# git push origin -f run_layer_alignment_v1

In [31]:
########### REGISTER ALGORITHM ON NEW ADE ##############
maap.register_algorithm_from_yaml_file("/projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Hunka_2024_NSD/Layer_Alignment/run_LA_ADE.yml").text

'{"code": 200, "message": {"id": "afdaec1e9b46b9dc41743d1d7cad468401c2d047", "short_id": "afdaec1e", "created_at": "2025-01-23T19:52:42.000+00:00", "parent_ids": ["0c613c249793f0fc585991920982311e223a31a2"], "title": "Registering algorithm: run_layer_alignment", "message": "Registering algorithm: run_layer_alignment", "author_name": "root", "author_email": "root@f2a98435594f", "authored_date": "2025-01-23T19:52:42.000+00:00", "committer_name": "root", "committer_email": "root@f2a98435594f", "committed_date": "2025-01-23T19:52:42.000+00:00", "trailers": {}, "extended_trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/afdaec1e9b46b9dc41743d1d7cad468401c2d047", "stats": {"additions": 7, "deletions": 7, "total": 14}, "status": "pending", "project_id": 3, "last_pipeline": {"id": 15073, "iid": 1856, "project_id": 3, "sha": "afdaec1e9b46b9dc41743d1d7cad468401c2d047", "ref": "hysds-v5", "status": "running", "source": "push", "created_at": "2025-01-23T1

#### RUN ALGORITHM FROM TERMINAL / SUBMIT JOBS TO DPS FOR ALL TILES ####

In [32]:
Desired_output_folder = '/projects/my-public-bucket/Data/Hunka_et_al_PAPER_NSD/JRC_TFM/'

TILES = [file for file in os.listdir(Desired_output_folder) if file.startswith("JRC_TMST")]
TILES = TILES[40:41]

split_string = "TMST_"
outfile_string = "_JRC_Transition_Map.tif"

i = 0
count = -1
submit_results_df_list = []
len_input_list = len(TILES)
list_jobIDs = [""]*len(TILES)

for TILE in TILES: 
    for TILE in TILES: 
        print(TILE)
        outfile_name = TILE.split(split_string)[1].split(".tif")[0]
        OUTFILE_NAME = outfile_name+outfile_string
        if not os.path.exists(os.path.join(Desired_output_folder,OUTFILE_NAME)): 
            print("submitting " + TILE)
            count +=1
            submit_result = maap.submitJob(identifier="run_layer_alignment", 
                           algo_id="run_layer_alignment",
                           version="run_layer_alignment_v1",
                           username="nehajo88",
                           queue="maap-dps-worker-32gb",
                           input_file="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Hunka_et_al_PAPER_NSD/JRC_TFM/"+TILE,
                           split_string = split_string,
                           outfile_string = outfile_string)
            job_id = str(submit_result).split(': ')[1].split(',')[0].replace("'","")
            print(job_id)
            time.sleep(1)

JRC_TMST_10S_040E.tif
submitting JRC_TMST_10S_040E.tif
75906458-6410-436d-a2c3-cabdd421015c


### ASSESS DPS RESULTS 

In [ ]:
Desired_output_folder = '/projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_COG/IPCC_classes/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)
SUMMARY_files_tifs = glob.glob('/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v8/IPCC_classes_ADE/2024/**/*IPCC*.tif', recursive=True)
print("Moving files...")
for each_file in SUMMARY_files_tifs:
    if os.path.exists(each_file) and not os.path.exists(os.path.join(Desired_output_folder,os.path.basename(each_file))): 
        print(each_file)
        shutil.move(each_file, Desired_output_folder)

Desired_output_folder = '/projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_COG/Age_classes/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)
SUMMARY_files_tifs = glob.glob('/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v8/IPCC_classes_ADE/2024/**/*Age*.tif', recursive=True)
print("Moving files...")
for each_file in SUMMARY_files_tifs:
    if os.path.exists(each_file) and not os.path.exists(os.path.join(Desired_output_folder,os.path.basename(each_file))): 
        print(each_file)
        shutil.move(each_file, Desired_output_folder)